In [ ]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [ ]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [ ]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

In [ ]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

In [ ]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

In [ ]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

# Practice

In [ ]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [ ]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as shape
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

# 1.5 Iterrow Approach

In [ ]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [ ]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    # Your code here

### 2.2 apply it to one of the focal crimes

### 2.3 Use apply to cover all the other focal crimes